# Delta Lake in Telecom Domain: Call Detail Records (CDR)

In this notebook, we will monitor call detail records in a telecom environment using managed and external Delta tables.

### Step 1: Create Schema

In [ ]:

# Create a schema for telecom data
spark.sql("""CREATE SCHEMA IF NOT EXISTS telecom_default_location;""")


### Step 2: Describe Schema

In [ ]:

# Describe the schema
spark.sql("""DESCRIBE SCHEMA EXTENDED telecom_default_location;""")


### Step 3: Create Managed Table

In [ ]:

# Create a managed table to store call detail records (CDR)
spark.sql("""
USE telecom_default_location;

CREATE OR REPLACE TABLE managed_table (
    call_id INT,
    caller_id STRING,
    recipient_id STRING,
    call_duration INT,
    call_time TIMESTAMP
);

INSERT INTO managed_table VALUES 
(1, '12345', '67890', 300, '2024-09-12 10:00:00'),
(2, '54321', '09876', 150, '2024-09-12 10:15:00');

SELECT * FROM managed_table;
""")


### Step 4: Describe Managed Table

In [ ]:

# Describe the managed table to get more information
spark.sql("""DESCRIBE EXTENDED managed_table;""")


### Step 5: Check Table Location

In [ ]:

# Python code to get the location of the managed table
tbl_location = spark.sql("DESCRIBE DETAIL managed_table").first().location
print(tbl_location)
files = dbutils.fs.ls(tbl_location)
display(files)


### Step 6: Drop the Managed Table

In [ ]:

# Drop the managed table
spark.sql("DROP TABLE managed_table;")


### Step 7: Create External Table

In [ ]:

# Create an external table using the provided CSV data
spark.sql("""
USE telecom_default_location;

CREATE OR REPLACE TEMPORARY VIEW temp_call_delays
USING CSV 
OPTIONS (
    path = '/mnt/data/telecom_call_delays.csv',
    header = "true",
    mode = "FAILFAST"
);

CREATE OR REPLACE TABLE external_table
LOCATION '/mnt/data/external_table'
AS SELECT * FROM temp_call_delays;

SELECT * FROM external_table;
""")


### Step 8: Describe External Table

In [ ]:

# Describe the external table
spark.sql("DESCRIBE EXTENDED external_table;")


### Step 9: Drop the External Table

In [ ]:

# Drop the external table
spark.sql("DROP TABLE external_table;")


### Step 10: Check External Data Location

In [ ]:

# Python code to check if the external data remains after dropping the table
tbl_path = "/mnt/data/external_table"
files = dbutils.fs.ls(tbl_path)
display(files)


### Step 11: Clean Up by Dropping the Schema

In [ ]:

# Drop the schema to clean up
spark.sql("DROP SCHEMA telecom_default_location CASCADE;")
